### Lab 3:  Variables and Templates


[From Ansible doc](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html#define-variables-in-inventory)

Using Variables
Ansible uses variables to manage differences between systems. With Ansible, you can execute tasks and playbooks on multiple different systems with a single command. To represent the variations among those different systems, you can create variables with standard YAML syntax, including lists and dictionaries. You can define these variables in your playbooks, in your inventory, in re-usable files or roles, or at the command line. You can also create variables during a playbook run by registering the return value or values of a task as a new variable.

After you create variables, either by defining them in a file, passing them at the command line, or registering the return value or values of a task as a new variable, you can use those variables in module arguments, in conditional “when” statements, in templates, and in loops. [The ansible-examples github repository](https://github.com/ansible/ansible-examples) contains many examples of using variables in Ansible.

Once you understand the concepts and examples on this page, read about Ansible facts, which are variables you retrieve from remote systems. With Ansible you can retrieve or discover certain variables containing information about your remote systems or about Ansible itself. Variables related to remote systems are called facts. With facts, you can use the behavior or state of one system as configuration on other systems. For example, you can use the IP address of one system as a configuration value on another system. Variables related to Ansible are called magic variables.

## Creating valid variable names
Not all strings are valid Ansible variable names. A variable name can only include letters, numbers, and underscores. Python keywords or playbook keywords are not valid variable names. A variable name cannot begin with a number.

Variable names can begin with an underscore. In many programming languages, variables that begin with an underscore are private. This is not true in Ansible. Variables that begin with an underscore are treated exactly the same as any other variable. Do not rely on this convention for privacy or security.

[Defining Variables](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html#define-variables-in-inventory)

## Variables

For this lab, we will leverage simple variables to be later used in a task. The task will append some content to a file and substitute the relevant variables.

### Start by defining the variable files:

<span style="color:red">#### Update the relevant field in the variables_local.yml file definition by editing the  next cell</span>.


In [ ]:
# Define the local and target variable files:
mkdir -p vars
cat > vars/variables_local.yml <<EOF
LABSTUDENTNAME: "INPUT YOUR NAME"
LABSTUDENTID: "{{ STDID }}"
LABLOCATION:  "INPUT YOUR COUNTRY"
EOF

cat > vars/variables_target.yml <<EOF
LABSTUDENTNAME: "APOLLO"
LABSTUDENTID: "11"
LABLOCATION: "THE MOON"
EOF


echo "########################"
echo "variable files created"
echo "########################"
echo " "
echo " "

#### Now create the playbook with the task

In [ ]:
# we use the existing invetory file :  no need to regenerate it

# Define a play

cat > play1.yml << EOF
- hosts: localhost
  gather_facts: true
  tasks:
    - include_vars: vars/variables_local.yml
    - name: Adapt lab3.1.txt file
      blockinfile:
        create: yes
        path: /student/student{{ STDID }}/student{{ STDID }}-lab3.1.txt
        block: |
          ##############################################################################################################################################################################
          # This file was generated by the [Ansible 101 {{ BRANDINGWOD }} Workshop-on-Demand](https://developer.hpe.com/hackshack/workshop/31) lab3 for {{ '{{' }} LABSTUDENTNAME {{ '}}' }} with id {{ '{{' }} LABSTUDENTID {{ '}}' }}  running from {{ '{{' }} hostvars[inventory_hostname]['ansible_facts']['nodename'] {{ '}}' }}
          ##############################################################################################################################################################################
EOF

echo "########################"
echo "play1.yml file created"
echo "########################"
echo " "
echo " "

Now let's execute the playbook on the inventory file we have defined earlier.

You will note that for now, the task is targetting the localhost only. The point is to show the principle here.

In [ ]:
ansible-playbook -i inventory play1.yml

### Now take a look at the result

In [ ]:
cat /student/student{{ STDID }}/student{{ STDID }}-lab3.1.txt

## Templates

In this part, we will leverage a template file (jinja Template with j2 extension) that will be used to create the final file.

We will then introduce conditions to alter the content of the file depending on the target. We will use the [ansible template module](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/template_module.html) to achieve this

In [ ]:
# Define a play

cat > play2.yml << EOF
- hosts: target
  gather_facts: true
  tasks:
    - include_vars: vars/variables_local.yml
    - name: Template a file to lab3.2.txt
      template:
        src: templates/lab3.2.txt.j2
        dest: /student/student{{ STDID }}/student{{ STDID }}-lab3.2.txt
        mode: '0644'
EOF

echo "########################"
echo "play2.yml file created"
echo "########################"
echo " "
echo " "

Let's look at the jinja template now:

In [ ]:
cat templates/lab3.2.txt.j2

### Execute the play now

In [ ]:
ansible-playbook -i inventory play2.yml

### Let's look at the results

In [ ]:
cat /student/student{{ STDID }}/student{{ STDID }}-lab3.2.txt
ssh {{  hostvars[inventory_hostname]['IP-WKSHP-Ansible101'] }} cat /student/student{{ STDID }}/student{{ STDID }}-lab3.2.txt

### What about making it into a playbook for next use in ansible?

In [ ]:
# Define a play

cat > result.yml << EOF
- hosts: target
  gather_facts: true
  tasks:
    - name: Check result for each host 
      command: cat /student/student{{ STDID }}/student{{ STDID }}-lab3.2.txt
      register: res
      
    - debug:
        msg="{{ '{{' }} res.stdout_lines {{ '}}' }}"

EOF

echo "########################"
echo "result.yml file created"
echo "########################"
echo " "
echo " "

### Let's check again

In [ ]:
ansible-playbook -i inventory result.yml

## Conditions

Use the when condition to control whether a task or role runs or is skipped. This is normally used to change play behavior based on facts from the destination system.

In our case, we will use the when condition on the targets to modify the content of the file based on variables set upper.

In [ ]:
# Define a play

cat > play3.yml << EOF
- hosts: target
  gather_facts: true
  tasks:
    - include_vars: vars/variables_local.yml
      when:  hostvars[inventory_hostname]['inventory_hostname']  == "127.0.0.1"
    - include_vars: vars/variables_target.yml
      when:  hostvars[inventory_hostname]['inventory_hostname']  != "127.0.0.1"
      
    - name: Template a file to lab3.3.txt
      template:
        src: templates/lab3.3.txt.j2
        dest: /student/student{{ STDID }}/student{{ STDID }}-lab3.3.txt
        mode: '0644'
EOF

echo "########################"
echo "play3.yml file created"
echo "########################"
echo " "
echo " "

Check the jinja template now

In [ ]:
cat templates/lab3.3.txt.j2

### Execute Now

In [ ]:
ansible-playbook -i inventory play3.yml

### And Check result using another result.yml play

In [ ]:
# Define a play

cat > result2.yml << EOF
- hosts: target
  gather_facts: true
  tasks:
    - name: Check result for each host 
      command: cat /student/student{{ STDID }}/student{{ STDID }}-lab3.3.txt
      register: res

    - debug:
        msg="{{ '{{' }} res.stdout_lines {{ '}}' }}"


EOF

echo "########################"
echo "result2.yml file created"
echo "########################"
echo " "
echo " "

In [ ]:
ansible-playbook -i inventory result2.yml

Let's see now a simple condition usecase leveraging the Operating System condition :  when Ubuntu do this and when Centos, do that.

In [ ]:
# Define a play

cat > play3.1.yml << EOF
- hosts: target
  gather_facts: true
  tasks:
  - name: "get curl package version"
    shell: rpm -q curl
    when: (ansible_distribution == "CentOS")
    register: curlc_version  

  - name: "get wget package version"
    shell: dpkg -s curl | grep Version
    when: (ansible_distribution == "Ubuntu")
    register: curlu_version  

  - name: "print curl version Centos"
    when: (ansible_distribution == "CentOS")
    debug:
      var: curlc_version.stdout

  - name: "print curl version Ubuntu"
    when: (ansible_distribution == "Ubuntu")
    debug:
      var: curlu_version.stdout
EOF
  
echo "########################"
echo "play3.1.yml file created"
echo "########################"
echo " "
echo " "

### Execute Now

In [ ]:
ansible-playbook -i inventory play3.1.yml

## What about looping now?

How about retrieving a serie of cover books from a publicly available API ? 
The API endpoiint will be the following: https://gutendex.com/books/


The playbook is defined as follows:
* We start by defining min max for the range of book covers we want to retrieve
* We leverage the loop feature to:
 * Retrieve book covers using the range from the API url " - name: "get online infos for books"
 * Extract using jq parsing the url of each covers and store them in a table | jq -r '.formats  | to_entries | .[] | select(.key=="image/jpeg").value'
 * Finally retrieve the picture from the url and copy it into the Pictures folder. 
 
 
### You can edit the playbook to change the BOOKMIN And BOOKMAX variables

In [ ]:
# Define a play

# Now looping to get the cover images from selected books in BOOKMIN AND BOOKMAX range.



# Set Book id Range you want to retrieve


cat > play3.2.yml << EOF
- hosts: localhost
  gather_facts: false
  tasks:
  - name: Define BOOKMIN
    set_fact:
      BOOKMIN: 20
    when: BOOKMIN is not defined

  - name: Define BOOKMAX
    set_fact:
      BOOKMAX: 25
    when: BOOKMAX is not defined
    
  - name: "get online infos for books"
    shell: curl -s https://gutendex.com/books/{{ '{{' }} item {{ '}}' }}/ | jq -r '.formats  | to_entries | .[] | select(.key=="image/jpeg").value'
    register: books_cover
    loop: "{{ '{{' }} range( BOOKMIN|int, BOOKMAX|int )|list {{ '}}' }}"
  
#  - name: debug
#    debug:
#      var: books_cover.results
      
  - name: "store covers in pictures folder"
    command: "curl -sS -o ./Pictures/{{ '{{' }} item.stdout | basename {{ '{{' }} {{ '{{' }} item.stdout {{ '{{' }}"
    loop: "{{ '{{' }} books_cover.results {{ '{{' }}"
  
EOF
  
echo "########################"
echo "play3.2.yml file created"
echo "########################"
echo " "
echo " "

## Please now check that the Pictures folder contain the result of the playbook

## Vault and Secret ??
Ansible handles secrets using a feature called Vault.
Vault lets you encrypt any of your .yml files, but typically you would apply it to files containing variable definitions, then use the variables’ values as needed elsewhere.

Vault provides subcommands that let you encrypt a file in place, decrypt a file in place, edit a file that’s encrypted in one step, etc.

When ansible is running your playbook or whatever, any time it comes across a .yml file that appears to be encrypted, it will decrypt it (in memory) and use the decrypted contents, fairly transparently. You can have as many of your files encrypted as you want.

As part of the workshop you were allocated with a student id and a password.
You are student{{ STDID }} and your password is {{ PASSSTU }}

We also provide your student with a dedicated url http:{{ JPHOSTEXT }}:{{ hostvars[inventory_hostname]['HTTPPORT-WKSHP-Ansible101'] }}/student{{ STDID }}
By default, the page is only accessible to your student along with its provided credenti
So as an example, why couldn't we encryt your student password ? 

* Let's start by creating a vault secret
* let's encrypt a variable and store in a yml file
* include the yml file in a playbook to benefit from the encrypted variable
* use the encrypted variable in playbook

# The website provides a generic url publicly available at http:{{ JPHOSTEXT }}:{{ hostvars[inventory_hostname]['HTTPPORT-WKSHP-Ansible101'] }}

Then your student has its own private zone under its student id folder. lease , check now the availability of your  private page now using a simple curl command. The page is password protected  and can only be opened with your own student credentials.

In [ ]:
curl --user student{{ STDID }}:{{ PASSSTU }} {{ JPHOSTEXT }}:{{ hostvars[inventory_hostname]['HTTPPORT-WKSHP-Ansible101'] }}/student{{ STDID }}/

This works great but as you can see, the password is provide in clear text in the command. Ansible provides us with a secure way  to connect to the same page leveraging ansible vault secret.

In [ ]:
# let's create a ply to check the default url
cat > play3.3.yml << EOF
- hosts: localhost
  gather_facts: false
  tasks:  
    - name: Check that you can connect (GET) to the default page and it returns a status 200
      uri:
        url: http://{{ JPHOSTEXT }}:{{ hostvars[inventory_hostname]['HTTPPORT-WKSHP-Ansible101'] }}
EOF
echo "########################"
echo "play3.3.yml file created"
echo "########################"
echo " "
echo " "

In [ ]:
ansible-playbook play3.3.yml

In [ ]:
# Create  your vault secret Now
ansible-vault encrypt_string --vault-password-file vault_secret {{ PASSSTU }} --name "'WEBLOGIN'" >> "var.yml

In [ ]:
# Now reach out the Web page with your login password
# let's create a ply to check the default url
cat > play3.4.yml << EOF
- hosts: localhost
  gather_facts: false
  include: var.yml
  tasks:  
    - name: Check that you can connect (POST) to your private page and it returns a status
      uri:
        url: http://{{ JPHOSTEXT }}:{{ hostvars[inventory_hostname]['HTTPPORT-WKSHP-Ansible101'] }}/student{{ STDID }}/
        user: student{{ STDID }}
        password: {{ '{{' }} WEBLOGIN {{ '}}' }}
        method: POST
        force_basic_auth: yes
        status_code: 201
EOF
echo "########################"
echo "play3.4.yml file created"
echo "########################"
echo " "
echo " "

In [ ]:
# Now perform the same action using curl and a ansible variable (Encrypted)

In [ ]:
ansible-playbook play3.4.yml

## Summary:
In this lab, we discovered variables and templates, conditions, loop and vault secret.
Let's now move on to the next topic: Roles.

* [Lab 4](4-WKSHP-Ansible101-Roles.ipynb)